In [4]:
# Library Load Model
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Library Pre-Processing
from nltk.stem import WordNetLemmatizer
import nltk
import re
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# loading token
with open('t.pickle', 'rb') as file_1:
    t = pickle.load(file_1)

In [6]:
# Load Model
  
model_gruimp = load_model('best_model.h5')

In [7]:
model_gruimp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 106, 128)          6450944   
                                                                 
 bidirectional (Bidirectiona  (None, 106, 64)          31104     
 l)                                                              
                                                                 
 gru_1 (GRU)                 (None, 8)                 1776      
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 6,483,833
Trainable params: 0
Non-trainable params: 6,483,833
______________________________________________

In [8]:
# Adding spanish to stopwords
spanish = stopwords.words('spanish')
additional_stopwords = []
for i in spanish:
    additional_stopwords.append(i)

# Adding french to stopwords
french = stopwords.words('french')
for i in french:
    additional_stopwords.append(i)

# Adding german to stopwords
german = stopwords.words('german')
for i in german:
    additional_stopwords.append(i)

In [9]:
# Setting stopwords with english as default language
stopwords = list(set(stopwords.words('english')))
for i in additional_stopwords:
    stopwords.append(i)

In [10]:
lemmatizer = WordNetLemmatizer()
def text_processing(text):

    # Converting all text to Lowercase
    text = text.lower()

    # Removing Unicode Characters
    text = re.sub("&#[A-Za-z0-9_]+", " ", text)

    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
  
    # Removing Whitespace
    text = text.strip()

    # Removing emoji
    text = re.sub("[^A-Za-z\s']", " ", text)

    # Removing double space
    text = re.sub("\s\s+" , " ", text)
        
    # Tokenizing words
    tokens = word_tokenize(text)

    # Removing Stopwords
    text = ' '.join([word for word in tokens if word not in stopwords])

    # Lemmatizer
    text = lemmatizer.lemmatize(text)

    return text

In [11]:
df_inf = pd.read_csv('h8dsft_p2m2_Ahmad Luay Adnani_inference.csv')
df_inf_copy = df_inf.copy()
df_inf_copy = df_inf.drop(['Unnamed: 0'],axis=1)
df_inf_copy.class_index.replace({1:'Negative Review',2:'Positive Review'}, inplace=True)

In [12]:
df_inf_copy

,class_index,review_title,review_text
0,Positive Review,"This Or ""Dixie Chicken"" Presents Them At A Peak",Though lyrically the overall feel of this reco...
1,Positive Review,"It's not perfect, but very good!",I have been looking for an anthology of outsta...
2,Positive Review,Well done,While the characters are not the same as in th...
3,Negative Review,"Not so great, but pretty colors",I tried/own both the pink and the orangish col...
4,Negative Review,Ruined My Valentine's Day!,I read each Amazon review on this product befo...
5,Positive Review,Concise Text of Neuroscience,I bought this book for my brother as he is tak...
6,Positive Review,GREAT!!!!!,I got this toy a couple of days ago and I ABSO...
7,Positive Review,Hot Coffee,Good coffee pot. Hot coffee. 6-8 cups makes a ...
8,Positive Review,Mystery at Walt Disney World,"Book arrived in good condition, but took quite..."
9,Negative Review,Take with a dose of skepticism...,"All readers should keep in mind that, as anoth..."


In [13]:
# Applying all preprocessing in one document

df_inf_copy['review_processed'] = df_inf_copy['review_text'].apply(lambda x: text_processing(x))
df_inf_copy

,class_index,review_title,review_text,review_processed
0,Positive Review,"This Or ""Dixie Chicken"" Presents Them At A Peak",Though lyrically the overall feel of this reco...,though lyrically overall feel record slightly ...
1,Positive Review,"It's not perfect, but very good!",I have been looking for an anthology of outsta...,looking anthology outstanding literary materia...
2,Positive Review,Well done,While the characters are not the same as in th...,characters original casting well done players ...
3,Negative Review,"Not so great, but pretty colors",I tried/own both the pink and the orangish col...,triedown pink orangish colors work way true go...
4,Negative Review,Ruined My Valentine's Day!,I read each Amazon review on this product befo...,read amazon review product made purchase full ...
5,Positive Review,Concise Text of Neuroscience,I bought this book for my brother as he is tak...,bought book brother taking class using book lo...
6,Positive Review,GREAT!!!!!,I got this toy a couple of days ago and I ABSO...,got toy couple days ago absolutely love much r...
7,Positive Review,Hot Coffee,Good coffee pot. Hot coffee. 6-8 cups makes a ...,good coffee pot hot coffee cups makes strong c...
8,Positive Review,Mystery at Walt Disney World,"Book arrived in good condition, but took quite...",book arrived good condition took quite bit lon...
9,Negative Review,Take with a dose of skepticism...,"All readers should keep in mind that, as anoth...",readers keep mind another reviewer noted arno ...


In [14]:
# Transform Inference-Set 
df_inf_transform = df_inf_copy.review_processed

In [15]:
word_index = t.word_index

In [16]:
word_index

{'<UNK>': 1,
 'time': 2,
 'movie': 3,
 'well': 4,
 'even': 5,
 'love': 6,
 'use': 7,
 'product': 8,
 'bought': 9,
 'better': 10,
 'cd': 11,
 'album': 12,
 'work': 13,
 'little': 14,
 'story': 15,
 'think': 16,
 'best': 17,
 'make': 18,
 'two': 19,
 'know': 20,
 'music': 21,
 'people': 22,
 'got': 23,
 'still': 24,
 'want': 25,
 'find': 26,
 'never': 27,
 'books': 28,
 'ive': 29,
 'money': 30,
 'recommend': 31,
 'see': 32,
 'made': 33,
 'didnt': 34,
 'years': 35,
 'found': 36,
 'go': 37,
 'cant': 38,
 'bad': 39,
 'used': 40,
 'say': 41,
 'another': 42,
 'ever': 43,
 'quality': 44,
 'put': 45,
 'reading': 46,
 'songs': 47,
 'thing': 48,
 'life': 49,
 'every': 50,
 'doesnt': 51,
 'thought': 52,
 'game': 53,
 'lot': 54,
 'easy': 55,
 'give': 56,
 'worth': 57,
 'sound': 58,
 'however': 59,
 'something': 60,
 'long': 61,
 'dvd': 62,
 'going': 63,
 'film': 64,
 'hard': 65,
 'last': 66,
 'since': 67,
 'characters': 68,
 'looking': 69,
 'need': 70,
 'right': 71,
 'year': 72,
 'day': 73,
 'anyon

In [17]:
df_inf_transform = t.texts_to_sequences(df_inf_transform)

In [18]:
# Padding the dataset to a maximum review length in words

df_inf_transform = pad_sequences(df_inf_transform, maxlen=106) # max len = 106 words

In [19]:
df_inf_transform

array([[    0,     0,     0, ...,    71,   636,  2810],
       [    0,     0,     0, ...,  5078,   649,   173],
       [    0,     0,     0, ...,     1,  1485,  3491],
       ...,
       [    0,     0,     0, ...,     1, 35808,   468],
       [    0,     0,     0, ...,   936,   276,  1531],
       [    0,     0,     0, ...,     1, 15530,   716]], dtype=int32)

In [20]:
# Predict using selected model
y_pred_inf = model_gruimp.predict(df_inf_transform)
y_pred_inf = np.where(y_pred_inf >= 0.5, 1, 0)
y_pred_inf

1/1 [==============================] - 1s 1s/step


array([[0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0]])

In [21]:
# Creating dataframe prediction
y_pred_inf = pd.DataFrame(y_pred_inf, columns=['Prediction'],index=df_inf_copy.index)

# Comparing the real target and prediction
model_inf = pd.concat([df_inf_copy['class_index'], y_pred_inf['Prediction']], axis=1).sort_index()
model_inf

,class_index,Prediction
0,Positive Review,0
1,Positive Review,0
2,Positive Review,1
3,Negative Review,1
4,Negative Review,0
5,Positive Review,1
6,Positive Review,1
7,Positive Review,1
8,Positive Review,1
9,Negative Review,0


In [22]:
model_inf.Prediction.replace({0:'Negative Review',1:'Positive Review'}, inplace=True)

In [23]:
model_inf

,class_index,Prediction
0,Positive Review,Negative Review
1,Positive Review,Negative Review
2,Positive Review,Positive Review
3,Negative Review,Positive Review
4,Negative Review,Negative Review
5,Positive Review,Positive Review
6,Positive Review,Positive Review
7,Positive Review,Positive Review
8,Positive Review,Positive Review
9,Negative Review,Negative Review


The model we created has 70% accuracy for this data inference, this might be due to the presence of new words that are not in the vocabulary.